# Final Full Chess Game

# Importing Libraries

In [2]:
import math
import operator
import sys
import pickle
from collections import defaultdict
import tensorflow as tf
from glob import glob
import random
import string
import serial
import struct
from time import sleep
import h5py
import cython
from image_slicer import slice
    
import cv2
load = cv2.imread
save = cv2.imwrite
    
import scipy.spatial as spatial
import scipy.cluster as clstr
    
import sklearn
    
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
    
from tensorflow import keras 
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.models import load_model
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
keras.backend.set_learning_phase(0)
    
import chess
import chess.engine
import chess.svg
# import chess.uci
    
from misc.config import *
from misc.utils import ImageObject, save_corners
from misc.slid import pSLID, SLID, slid_tendency

# from tensorflow.keras.optimizers import RMSprop
from misc.laps import LAPS                       
from misc.llr import LLR, llr_pad 

from misc import utils

from IPython.core.interactiveshell import InteractiveShell

## Constants Defined

In [14]:
HEIGHT = 150
WIDTH = 150
IMAGE_DIMS = (150, 150)

MODEL_LOC1 = '/misc/data/models/laps.h5'
MODEL_LOC2 = "/Model/saved_models/trained_model_final_bs32_225_dg_6.h5"

CHESS_ENGINE_PATH = "F:/Github/stockfish_14.1_win_x64_avx2/stockfish_14.1_win_x64_avx2.exe"

ARDUINO_PORT = 'COM4'

IP = 'http://192.168.29.227:8080/video'

## Utility Functions

In [ ]:
def show_img(img):
    img = cv2.resize(img, (700, 700))
    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
        
def adjust_gamma(image, gamma=1.5):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    
    return cv2.LUT(image, table)

# Board Array and FEN Notation Generation

In [ ]:
def generate_board_arr1(img_number):
        
    row = ["H", "G", "F", "E", "D", "C", "B", "A"]
    col = ["8", "7", "6", "5", "4", "3", "2", "1"]
        
    a = 1
    b = 1
        
    matrix = []

    for r in range(0, 8):
        matrix.append([0 for c in range(0, 8)])

        
    arr = np.zeros((8, 8))
        
    an_array = np.full([8, 8], None)
        
    slice("chess-playing-robot/During Game/04 Picture Selected From Model 1/"+str(img_number)+".png" , 64)
        
    for j in row:
        b=1
        for k in col:
            os.rename("chess-playing-robot/During Game/04 Picture Selected From Model 1/"+str(img_number)+"_0" + str(a) + "_0"+ str(b) +".png","chess-playing-robot/During Game/05 Model 2 Output/" + j + k +".png")
            b += 1
        a += 1
        
    a = 0
    b = 0
        
    row = ["A", "B", "C", "D", "E", "F", "G", "H"]
    col = ["8", "7", "6", "5", "4", "3", "2", "1"]
        
    for j in col:
        a = 0
        for k in row:
            img = image.load_img("chess-playing-robot/During Game/05 Model 2 Output/" + k + j + ".png", target_size=(HEIGHT, WIDTH))
                
            img = np.reshape(img,(1,150,150,3))
            # print(img.shape)

            y = model1.predict_classes(img)
            print(y)

            if y==0:
                classes='E'
            elif y==1:
                classes='B'
            elif y==2:
                classes='K'
            elif y==3:
                classes='N'
            elif y==4:
                classes='P'
            elif y==5:
                classes='Q'
            elif y==6:
                classes='R'
            elif y==7:
                classes='b'
            elif y==8:
                classes='k'
            elif y==9:
                classes='n'
            elif y==10:
                classes='p'
            elif y==11:
                classes='q'
            else:
                classes='r'

            an_array[b][a] = classes
                
            a += 1
        b += 1
    return an_array

def fen_data_generation(new_board_arr):        
    fen=""
    count1=0
    count2=0
        
    for r in new_board_arr:
        s=""
        count1=count1+1;
        count2=0;
        for c in r:
            if c=='E':
                count2=count2+1;
            else:
                if count2!=0:
                    s=s+str(count2);
                s=s+c;
                count2=0;
        if count2!=0:
            s=s+str(count2);
        if count1!=1:
            fen=fen+'/'+s
        else:
            fen=fen+s
    fen=fen+' b - - 0 0'
    return fen

# Take Board Pictures and Pass Trough Model

In [ ]:
def saveBoardImage():
    a=0
    count=1
    cam = cv2.VideoCapture(IP)
       
    while count < 6:
        # Get current frame
        ret, img = cam.read()
        cv2.namedWindow('Board Image',cv2.WINDOW_NORMAL)
        cv2.imshow("Board Image", img)
        
        key = cv2.waitKey(1)

        if key % 256 == 27: #ESC Pressed
            break

        elif key % 56 == 32: #SPACE Pressed
            for a in range(0, 5):
                file='chess-playing-robot/During Game/01 Original Image/' + str(count) + '.png'

                cv2.imwrite(file, img)
                count += 1
                sleep(1)

    # Release the object
    cam.release()

    # Close the window
    cv2.destroyAllWindows()

    
def passSavedImageToModel():
    
    count = 1
    
    perceptLayer = PerceptionLayer(model)
    
    while count <= 5:
        board_img = cv2.imread("chess-playing-robot/During Game/01 Original Image/"+ str(count) + ".png")

        try:
            perceptLayer.detect(board_img)
        except:
            count += 1
            continue
        else:
            img = cv2.imread("chess-playing-robot/3.png")
            
            img = cv2.resize(img, (700, 700))
            cv2.imshow("Board Image", img)
            
            key = cv2.waitKey(0)
            
            if key % 256 == 27: #ESC Pressed
                print ("REJECTED")
                count += 1
                continue
        
            elif key % 56 == 32: #SPACE Pressed
                print ("ACCEPTED")
                os.rename("chess-playing-robot/1.png", "chess-playing-robot/During Game/02 Model 1 Output 1/1.png")
                os.rename("chess-playing-robot/2.png", "chess-playing-robot/During Game/03 Model 1 Output 2/1.png")
                os.rename("chess-playing-robot/3.png", "chess-playing-robot/During Game/04 Picture Selected From Model 1/1.png")
                count = 5
                break

        count += 1
        
    cv2.destroyAllWindows()
    

def rotateImage():
    
    os.rename("chess-playing-robot/During Game/04 Picture Selected From Model 1/1.png", "chess-playing-robot/During Game/04 Picture Selected From Model 1/2.png")
    path = r'chess-playing-robot/During Game/04 Picture Selected From Model 1/2.png'
    src = cv2.imread(path)
    image = cv2.rotate(src, cv2.ROTATE_180)
    
    file='chess-playing-robot/During Game/04 Picture Selected From Model 1/1.png'
    cv2.imwrite(file, image)
    os.remove("chess-playing-robot/During Game/04 Picture Selected From Model 1/2.png")

# Delete Files

In [ ]:
def deleteFiles():
    a = 0
    b = 0
        
    row = ["A", "B", "C", "D", "E", "F", "G", "H"]
    col = ["8", "7", "6", "5", "4", "3", "2", "1"]
        
    for j in col:
        a = 0
        for k in row:
            os.remove("chess-playing-robot/During Game/05 Model 2 Output/" + k + j + ".png")
    
    os.remove("chess-playing-robot/During Game/01 Original Image/1.png")
    os.remove("chess-playing-robot/During Game/01 Original Image/2.png")
    os.remove("chess-playing-robot/During Game/01 Original Image/3.png")
    os.remove("chess-playing-robot/During Game/01 Original Image/4.png")
    os.remove("chess-playing-robot/During Game/01 Original Image/5.png")
    
    os.remove("chess-playing-robot/During Game/02 Model 1 Output 1/1.png")
    os.remove("chess-playing-robot/During Game/03 Model 1 Output 2/1.png")
    os.remove("chess-playing-robot/During Game/04 Picture Selected From Model 1/1.png")

# Check Square Availability

In [ ]:
def checkDestAvailability(squareName):    
    
    img = image.load_img("chess-playing-robot/During Game/05 Model 2 Output/" + squareName + ".png", target_size=(HEIGHT, WIDTH))
    
    img = np.reshape(img,(1,150,150,3))

    y = model1.predict_classes(img)

    if y==0:
         return '0'
    else:
        return '1'

# Reading Angles From File

In [12]:
AllAngles = [None] * 0

with open("Arm Movement\Angles.txt", "r") as file:
 
    for line in file:
     
        for word in line.split():
            
            AllAngles.append(word)

## Getting Specific Angle 

In [ ]:
def getSouceAnglesFromFile(SquareName):
    
    count1 = 0
    
    Angle1 = 0 
    Angle2 = 0
    Angle3 = 0
    Angle4 = 0

    for word in AllAngles:
        if word == SquareName:
            count1 += 1
            continue

        if count1 == 1:
            Angle1 = word
            count1 += 1
            continue

        if count1 == 2:
            Angle2 = word
            count1 += 1
            continue

        if count1 == 3:
            Angle3 = word
            count1 += 1
            continue

        if count1 == 4:
            Angle4 = word
            count1 += 1
            continue
    ReqAngle = [Angle1, Angle2, Angle3, Angle4]
    
    return ReqAngle

## Perception Layer

### Core Functions

In [ ]:
class PerceptionLayer:

    def __init__(self, model):
        
        self.model = model

    def layer(self):
        global NC_LAYER, NC_IMAGE

        segments = pSLID(NC_IMAGE['main'])
        raw_lines = SLID(NC_IMAGE['main'], segments)
        lines = slid_tendency(raw_lines)
        points = LAPS(NC_IMAGE['main'], lines)
        inner_points = LLR(NC_IMAGE['main'], points, lines)
        four_points = llr_pad(inner_points, NC_IMAGE['main'])

        try:
            NC_IMAGE.crop(four_points)
            save(str(NC_LAYER) + ".png", NC_IMAGE['orig'])

        except:
            utils.warn("Next layer is not needed")
            NC_IMAGE.crop(inner_points)

        print("\n")


    def detect(self, image):
        global NC_LAYER, NC_IMAGE, NC_CONFIG

        NC_IMAGE, NC_LAYER = ImageObject(image), 0
        for _ in range(NC_CONFIG['layers']):
            NC_LAYER += 1
            self.layer()

        #show_img(NC_IMAGE['orig'])
        return NC_IMAGE['orig']

### Loading and Testing Model"

#### Prediction Functions

In [ ]:
def predict(model, img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return preds[0]
    
def plot_preds(img, preds):
    gs = gridspec.GridSpec(2, 1, height_ratios=[4, 1])
    plt.figure(figsize=(8, 8))
    plt.subplot(gs[0])
    plt.imshow(np.asarray(img))
    plt.subplot(gs[1])
    plt.barh(range(0, CLASSES), preds, alpha=0.5)
    plt.yticks(range(0, CLASSES), LABELS)
    plt.xlabel('Probability')
    plt.xlim(0, 1)
    plt.tight_layout()
        
def get_pred(preds, labels):
    index_of_max = np.argmax(preds)
    return labels[index_of_max]

#### Load Model

In [ ]:
model = tf.keras.models.load_model(MODEL_LOC1)
model1 = tf.keras.models.load_model(MODEL_LOC2)

# Loading Chess Engine

In [ ]:
# engine = chess.engine.SimpleEngine.popen_uci(CHESS_ENGINE_PATH)

## Controller Layer

In [ ]:
class ControllerLayer:

    def __init__(self, arduino):

        self.__arduino = arduino

    def send_to_arduino(self, capturing, angle1, angle2):

        self.__arduino.write(struct.pack('>BBBBBBBBB', int(capturing), int(angle1[0]), int(angle1[1]), int(angle1[2]),
                                         int(angle1[3]), int(angle2[0]), int(angle2[1]), int(angle2[2]),
                                         int(angle2[3])))

## Full Game Execution

In [ ]:
# Initalize Controller layer
arduino = serial.Serial(ARDUINO_PORT, 9600)
controlLayer = ControllerLayer(arduino)

# Initalize Perception Layer
perceptLayer = PerceptionLayer(model)

# Step By Step Execution

### Capturing Image And generation Fen Notation

In [ ]:
file_name = 1
capturing = 0
    
engine = chess.engine.SimpleEngine.popen_uci(CHESS_ENGINE_PATH)

saveBoardImage()
passSavedImageToModel()
rotateImage()
    
board_array = generate_board_arr1(file_name)

fen_notation = fen_data_generation(board_array)

board = chess.Board(fen_notation)

### Checking what the Model has predicte

In [ ]:
board_array

In [ ]:
chess.svg.board(board, size=350)

### Generating Counter Move

In [ ]:
result = engine.play(board, chess.engine.Limit(time=3))
print(result.move)

### Pushing counter Move on the Board

In [ ]:
board.push(result.move)
chess.svg.board(board, size=350)

### Getting the Squares Names

In [ ]:
squares = str(result.move)

Square = list(map(''.join, zip(*[iter(squares)]*2)))

### Getting angles of the two Square

In [ ]:
Angles1 = getSouceAnglesFromFile(Square[0].upper())
Angles1

In [ ]:
Angles2 = getSouceAnglesFromFile(Square[1].upper())
Angles2

### Checking if the destination square available

In [ ]:
capturing = checkDestAvailability(Square[1].upper())
print(capturing)

### Sending angles to aruino to execute the move

In [ ]:
deleteFiles()

controlLayer.send_to_arduino(capturing,Angles1,Angles2)
    
engine.quit()

In [ ]:
arduino.close()

# Full Game Execution in Loop

In [ ]:
file_name = 1
capturing = 0
    
# Initalize Perception Layer
perceptLayer = PerceptionLayer(model)

# sleep(5)

while True:
    
    engine = chess.engine.SimpleEngine.popen_uci(CHESS_ENGINE_PATH)

    saveBoardImage()
    passSavedImageToModel()
    rotateImage()

    board_array = generate_board_arr1(file_name)

    fen_notation = fen_data_generation(board_array)

    board = chess.Board(fen_notation)

    chess.svg.board(board, size=350)

    result = engine.play(board, chess.engine.Limit(time=3))
    squares = str(result.move)

    Square = list(map(''.join, zip(*[iter(squares)]*2)))

    Angles1 = getSouceAnglesFromFile(Square[0].upper())
    Angles2 = getSouceAnglesFromFile(Square[1].upper())

    capturing = checkDestAvailability(Square[1].upper())
    
    sleep(5)
    
    deleteFiles()

    controlLayer.send_to_arduino(capturing,Angles1,Angles2)
    
    engine.quit()
        

arduino.close()

# Dataset Creation Script

# Taking 5 images and Passing them to Model

In [ ]:
def takeBoardImage():
    
    a=0
    cam = cv2.VideoCapture(IP)
    count3 = 1

    while True:
        count=1
        count2 = 1
        while count < 6:
            # Get current frame
            ret, img = cam.read()
            cv2.namedWindow('Board Image',cv2.WINDOW_NORMAL)
            cv2.imshow("Board Image", img)

            key = cv2.waitKey(1)

            if key % 256 == 27: #ESC Pressed
                break

            elif key % 56 == 32: #SPACE Pressed
                for a in range(0, 5):
                    file='chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/' + str(count) + '.png'

                    cv2.imwrite(file, img)
                    count += 1
                    sleep(0.7)

        # Close the window
        cv2.destroyAllWindows()


        perceptLayer = PerceptionLayer(model)
        count2 = 1

        while count2 <= 5:
            board_img = cv2.imread("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/"+ str(count2) + ".png")

            try:
                perceptLayer.detect(board_img)
            except:
                count2 += 1
                continue
            else:
                img = cv2.imread("chess-playing-robot/3.png")

                img = cv2.resize(img, (700, 700))
                cv2.imshow("Board Image", img)

                key = cv2.waitKey(0)

                if key % 256 == 27: #ESC Pressed
                    print ("REJECTED")
                    count2 += 1
                    continue

                elif key % 56 == 32: #SPACE Pressed
                    print ("ACCEPTED")
                    # os.rename("chess-playing-robot/1.png", "chess-playing-robot/During Game/02 Model 1 Output 1/1.png")
                    # os.rename("chess-playing-robot/2.png", "chess-playing-robot/During Game/03 Model 1 Output 2/1.png")
                    os.rename("chess-playing-robot/3.png", "chess-playing-robot/Model/All new Dataset/train 5/02 Selected ChessBoard/"+str(count3)+".png")
                    count3 +=1
                    count2 = 5
                    break
                
            count2 += 1
        os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/1.png")
        os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/2.png")
        os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/3.png")
        os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/4.png")
        os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/5.png")
        cv2.destroyAllWindows()

In [ ]:
def takeBoardImage():
    
    a=0
    cam = cv2.VideoCapture(IP)
    count3 = 1

    while True:
        count2 = 1
        # Get current frame
        ret, img = cam.read()
        cv2.namedWindow('Board Image',cv2.WINDOW_NORMAL)
        cv2.imshow("Board Image", img)

        key = cv2.waitKey(1)

        if key % 256 == 27: #ESC Pressed
            break

        elif key % 56 == 32: #SPACE Pressed
            file='chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/' + str(count) + '.png'

            cv2.imwrite(file, img)

        # Close the window
        cv2.destroyAllWindows()


        perceptLayer = PerceptionLayer(model)

        board_img = cv2.imread("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/1.png")

        try:
            perceptLayer.detect(board_img)
        except:
            break
        else:
            img = cv2.imread("chess-playing-robot/3.png")

            img = cv2.resize(img, (700, 700))
            cv2.imshow("Board Image", img)

            key = cv2.waitKey(0)

            if key % 256 == 27: #ESC Pressed
                break

            elif key % 56 == 32: #SPACE Pressed
                print ("ACCEPTED")
                # os.rename("chess-playing-robot/1.png", "chess-playing-robot/During Game/02 Model 1 Output 1/1.png")
                # os.rename("chess-playing-robot/2.png", "chess-playing-robot/During Game/03 Model 1 Output 2/1.png")
                os.rename("chess-playing-robot/3.png", "chess-playing-robot/Model/All new Dataset/train 5/02 Selected ChessBoard/700 ("+str(count3)+").png")
                count3 +=1
                count2 = 5
                break
                
        os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/1.png")
        # os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/2.png")
        # os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/3.png")
        # os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/4.png")
        # os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/5.png")
        cv2.destroyAllWindows()
        
# def deleteFiles_1():
    
#     os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/1.png")
#     os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/2.png")
#     os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/3.png")
#     os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/4.png")
#     os.remove("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/5.png")

In [ ]:
file_name = 1

while file_name <= 194:
    
    slice("chess-playing-robot/Model/All new Dataset/train 5/Slices/700 (" + str(file_name) + ").png", 64)
    file_name += 1

In [ ]:
takeBoardImage()

In [ ]:
cam = cv2.VideoCapture(IP)
names = 1

while True:
    ret, board_img = cam.read()
    
    cv2.namedWindow('Board Image',cv2.WINDOW_NORMAL)
    cv2.imshow("Board Image", board_img)

    key = cv2.waitKey(1)
        
    if key % 256 == 27: #ESC Pressed
        break
        
    elif key % 56 == 32: #SPACE Pressed
            
        # board_img = adjust_image(board_img)
        # cv2.imwrite('img1.png', board_img)
        # show_img(board_img)
        save("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/"+ str(names) + ".png", board_img)
        names += 1

cam.release()
cv2.destroyAllWindows()

In [ ]:
perceptLayer = PerceptionLayer(model)
file_name = 56

while file_name <= 121:

    board_img = cv2.imread("chess-playing-robot/Model/All new Dataset/train 5/01 Original Pic/"+ str(file_name) + ".png")
    
    cv2.namedWindow('Board Image',cv2.WINDOW_NORMAL)
    cv2.imshow("Board Image", board_img)
    
    try:
        perceptLayer.detect(board_img)
    except:
        file_name += 1
        break
    else:
        # os.rename("chess-playing-robot/1.png", "chess-playing-robot/During Game/02 Model 1 Output 1/" + str(file_name) + ".png")
        # os.rename("chess-playing-robot/2.png", "chess-playing-robot/During Game/03 Model 1 Output 2/" + str(file_name) + ".png")
        os.rename("chess-playing-robot/3.png", "chess-playing-robot/Model/All new Dataset/train 5/02 Selected ChessBoard/" + str(file_name) + ".png")
        file_name += 1

cv2.destroyAllWindows()

In [ ]:
slice("chess-playing-robot/During Game/2.png",64)